In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import random
from datetime import datetime

#23 tags
tags = ['Data Science', 'Data Visualization', 'Data Analysis', 'Python', 'R', 'Machine Learning', 'Big Data',
        'Deep Learning', 'Data Engineering', 'Artificial Intelligence', 'Neural Networks', 'SQL', 'Data Mining', 
        'Web Scraping', 'Data Quality','Data Cleaning', 'Data Collection', 'Data Modeling', 
        'Data Storytelling', 'Software Engineering','Data Science Portfolio', 'Dashboard', 'Database']

tags_new = [t.replace(' ','-').lower() for t in tags]

data = []
url_template = "https://medium.com/tag/{0}/archive/2022/{1}/{2}"

def get_claps(claps_str):
    if (claps_str is None) or (claps_str == ''):
        claps = 0
    else:
        split = claps_str.split('K')
        claps = float(split[0])
        if len(split) == 2:
            claps = int(claps*1000)
        else:
            claps = int(claps)
    return claps

# Loop over the tags, months and days
for tag in tags_new:
    for i in range(100):
        month = str(random.randint(1, 12)).zfill(2)
        day = str(random.randint(1, 28)).zfill(2)
        # Try to retrieve the HTML content for the given tag, month, and day
        url = url_template.format(tag, month, day)
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        # Get all articles in the page
        articles = soup.find_all("div", class_="streamItem streamItem--postPreview js-streamItem")
        # Get all elements from the articles        
        for article in articles:
            title = article.find('h3').text if article.find('h3') else '-'
    
            author = article.find('a', class_='ds-link ds-link--styleSubtle link link--darken link--accent u-accentColor--textNormal u-accentColor--textDarken')
            author = author.text if author else None
    
            publication = article.find('a', class_='ds-link ds-link--styleSubtle link--darken link--accent u-accentColor--textNormal')
            publication = publication.text if publication else None
    
            claps = article.find('button', class_='button button--chromeless u-baseColor--buttonNormal js-multirecommendCountButton u-disablePointerEvents')
            claps = claps.text if claps else None
            claps = get_claps(claps)
    
            responses = article.find('a', class_='button button--chromeless u-baseColor--buttonNormal')
            responses = int(re.sub("[^0-9]", "", responses.text)) if responses else 0
    
            reading_time = article.find('span', class_='readingTime')
            reading_time = 0 if reading_time is None else int(reading_time['title'].split(' ')[0])
    
            date = article.find('time').text
            date = datetime.strptime(date, '%b %d, %Y')
            date = date.strftime('%Y-%m-%d')
    
            tags = tag 
    
            data.append([title, author, claps, responses, reading_time, publication, date, tag])
        
                
df = pd.DataFrame(data, columns=['title','author', 'claps', 'responses', 'reading time', 'publication', 'date', 'tag'])

df.to_csv('medium_articles_2022.csv', index=False)